In [2]:
#import libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from mpl_toolkits.mplot3d import Axes3D
import neurokit2 as nk
import random
path = "/scratch/alim/overnight_validation/MUSE-PSG/"
import os
import random
import pickle
import heartpy as hp
apnea_labels = ["Mixed Apnea", "Central Apnea", "Obstructive Apnea"]

In [3]:
MUSE_SAMPLING_RATE = 64
apnea_labels = ["Mixed Apnea", "Central Apnea", "Obstructive Apnea"]

'''
THE FOLLOWING LENGTH OF CODE LINES 17-83 PROCESSES FILES INTO A USEABLE PATIENT_DATA_DIRECTORY
'''
def read_files_in_directory(directory_path):
    try:
        #print("Finding all MUSE-PSG files")
        # Get a list of all files in the directory
        file_list = sorted([entry.name for entry in os.scandir(directory_path) if
                    entry.is_file() and any(keyword in entry.name for keyword in ['ppg','events'])])[0:2]


                     
        # Read the contents of each file
        #print("Findind all file names completed")

        return file_list
    except Exception as e:
        print(f"Error reading files: {e}")

# Example usage:
directory_path = "/scratch/alim/overnight_validation/MUSE-PSG"
file_name_list = read_files_in_directory(directory_path)

patient_dictionary = {}

#print("Organizing into dictionary")

for filename in file_name_list:
    parts = filename.rsplit('_',1)
    patient_name = parts[0]
    
    if patient_name in patient_dictionary:
        patient_dictionary[patient_name].append(filename)

    else:
        patient_dictionary[patient_name] = [filename]

#print("Finished Organizing into dicitonary")

#print("Short Preview of the Dictionary:")

def read_csv_to_dataframe(file_path):
    try:
        # Read the CSV file into a Pandas DataFrame
        dataframe = pd.read_csv(file_path)

        # Return the DataFrame
        return dataframe

    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None
    
patient_data_dictionary = {}
#read the csv for all patient files
for patient in patient_dictionary:
    files = patient_dictionary[patient]
    if len(files) != 2:
        continue
    patient_data_dictionary[patient] = {'events': read_csv_to_dataframe(path + files[0]),'ppg':read_csv_to_dataframe(path + files[1])}



#Change the time from unix format to regular date

for patient in patient_data_dictionary:

    patient_data_dictionary[patient]['ppg']['ts-datetime'] = pd.to_datetime(patient_data_dictionary[patient]['ppg']['ts'], unit='s')
    patient_data_dictionary[patient]['events']['start-datetime'] = pd.to_datetime(patient_data_dictionary[patient]['events']['start'], unit='s')
    patient_data_dictionary[patient]['events']['end-datetime'] = patient_data_dictionary[patient]['events']['start-datetime'] + pd.to_timedelta(patient_data_dictionary[patient]['events']['duration'], unit='s')

In [56]:
def check_nan_or_masked(data_dict):
    for value in data_dict.values():
        if value == "masked" or pd.isna(value):
            return True
    return False

In [4]:
matrix = np.empty ((0,10))
sample_rate = 64

dummy = patient_data_dictionary
for patient in dummy:
    events = dummy[patient]['events']
    ppg_signal = dummy[patient]['ppg']
    apnea = events[events['name'].isin(apnea_labels)]
    ppg_signal.set_index('ts-datetime', inplace=True)

    # Define the frequency for the intervals (e.g., '10S' for 10-second intervals)
    freq = '10S'

    # Group the DataFrame by the specified frequency
    ppg_10s_intervals = ppg_signal.groupby(pd.Grouper(freq=freq))

    # Now ppg_10s_intervals contains the DataFrame divided into 10-second intervals
    # You can access individual groups using the get_group method
    for interval, data in ppg_10s_intervals:
        ppg = data
        overlapping_apnea = apnea[(apnea['start-datetime'] <= interval  + pd.to_timedelta(10,unit = 's')) & (apnea['end-datetime'] >=interval)]
        # ppg['ch2'] = (ppg['ch2'] - ppg['ch2'].min(axis = 0))/(ppg['ch2'].max(axis=0)-ppg['ch2'].min(axis=0))
        ppg = ppg['ch2']
        try:
            label = 0
            wd,m = hp.process(ppg,sample_rate = sample_rate)

            #['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate']
        # Basic metrics
            print(m)
            heart_rate = m['bpm']  # Heart Rate in BPM
            ibi = m['ibi']  # Average Interbeat Interval
            sdnn = m['sdnn']  # Standard Deviation of NN intervals
            sdsd = m['sdsd']  # Standard Deviation of successive differences between adjacent NN intervals

            # HRV metrics
            rmssd = m['rmssd']  # Root Mean Square of Successive Differences
            pnn50 = m['pnn50']  # proportion of differences between adjacent NN intervals that are greater than 50ms

            std = np.std(ppg)
            mean = np.mean(ppg)
            var = np.var(ppg)



            if not overlapping_apnea.empty:

                label = 1

            feature_row = np.array([heart_rate,ibi,sdnn,sdsd,rmssd,pnn50,std,mean,var,label])
            feature_row = np.nan_to_num(feature_row,nan=-1)
            matrix = np.vstack([matrix,feature_row])



        except hp.exceptions.BadSignalWarning as e:
            pass
            #print("Bad signal quality. Consider checking the data or further preprocessing.")


        except:
            print('error')

#
print(matrix)
        








error


/home/heoyoun1/ENV/lib/python3.10/site-packages/heartpy/peakdetection.py:186: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peaksy = hrdata[peaksx]
/home/heoyoun1/ENV/lib/python3.10/site-packages/heartpy/peakdetection.py:201: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  working_data['ybeat'] = [hrdata[x] for x in peaklist]
/home/heoyoun1/ENV/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/heoyoun1/ENV/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  re

{'bpm': nan, 'ibi': nan, 'sdnn': nan, 'sdsd': masked, 'rmssd': nan, 'pnn20': nan, 'pnn50': nan, 'hr_mad': nan, 'sd1': nan, 'sd2': nan, 's': nan, 'sd1/sd2': nan, 'breathingrate': nan}
{'bpm': nan, 'ibi': nan, 'sdnn': nan, 'sdsd': masked, 'rmssd': nan, 'pnn20': nan, 'pnn50': nan, 'hr_mad': nan, 'sd1': nan, 'sd2': nan, 's': nan, 'sd1/sd2': nan, 'breathingrate': nan}
error
[[-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00  1.79650344e+07  1.42308333e+08
   3.22742462e+14  0.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00  2.51907925e+07  5.74121631e+07
   6.34576029e+14  0.00000000e+00]]


PLOTTING HEART_RATE



In [5]:
def plotFeatures(labels,features,fileName,featureName):
    plt.scatter(x = labels, y = features)
    plt.title(featureName)
    plt.ylabel(featureName)
    plt.xlabel('apnea = 1, non-apnea = 0')
    plt.savefig(fileName)
    plt.clf()


In [6]:
#plotting heart rate
labels = matrix[:,9]

featureNames = ['Heart Rate (BPM)','IBI','SDNN','SDSD','RMSSD','PNN50','STDS','MEANS','VARS']

for i in range(len(featureNames)):
    featureName = featureNames[i]

    features = matrix[:,i]

    plotFeatures(labels,features,f"./visualizations/{featureName}.png",featureName)



In [8]:
pd.DataFrame(matrix).to_csv('./matrix.csv',index = True)